<a href="https://colab.research.google.com/github/Ach113/Text-classification-using-neural-networks/blob/master/review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk.stem.snowball import SnowballStemmer
import string

def format_text(f):
    all_text = f.read()

    words = ""
    if len(all_text) > 1:
        # removing punctuation
        exclude = set(string.punctuation)
        text_string = ''.join(ch for ch in all_text if ch not in exclude)

        from nltk.tokenize import word_tokenize
        # tokenizing string into words
        words = word_tokenize(text_string)
        stemmer = SnowballStemmer('english')
        text = []
        # replacing words with their stems
        for word in words:
            text.append(stemmer.stem(word))
        # detokenizing the words
        text = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in text]).strip()

        return text

In [0]:
import os
import numpy as np
import tensorflow as tf

In [0]:
import nltk
nltk.download('punkt')

In [0]:
path = '/content/Data'
labels = ['neg', 'pos']

X = []
y = []

# count the procecced files and set limit to their amount
i = 0
limit = 5000

for label in labels:
  abs_path = os.path.join(path, label)
  for file in os.listdir(abs_path):
    if i==limit:
      break
    f = open(os.path.join(abs_path, file), 'r')
    # call function that formats the text by removing punctuation and stemming
    F = format_text(f)
    X.append(F)
    print('Extracting text #', i+1)
    i += 1
    if label == 'neg':
      y.append(1)
    elif label == 'pos':
      y.append(0)
  i = 0
print('Finished!')

In [0]:
# divide data into train/test parts
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle = True)

In [0]:
# turn text data into numbers via vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words = 'english')
x_train = vectorizer.fit_transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()

In [0]:
# normalization

x_train = tf.keras.utils.normalize(x_train)
x_test = tf.keras.utils.normalize(x_test)
y_train = np.array(y_train)

In [41]:
# Naive Bayes ML classifier for comparison
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(pred, y_test)
print('Accuracy score: ', score)

Accuracy score:  0.6756


In [0]:
from keras.models import Sequential
from keras import layers

In [0]:
# setting up the keras model

model = Sequential()
shape = x_train.shape[1]

model.add(layers.Dense(10, input_dim=shape, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 10)

In [0]:
# evaluating loss and accuracy
loss, accuracy = model.evaluate(x_test, np.array(y_test))
print(round(loss, 4), str(round(accuracy, 4)*100) + '%')